# TODO 

- Data from 2000-2010
- Hourly values of PV-Pot
    - Find min/max
    - Daily averages
    - Weekly averages
- Sum up over europes landmass
-  Rank it by the highest - Take x many highest cells that are needed
    - this will kind of already give the area 
- Take mean over Europe?? take mean over Gridcells in Europe?
    - 1000km sidelenght find minimum area? Is it consistent?
    - Get the means in the Gridcells hourly/weekly
    - Compare it throughout the years
          - Does the grid with pv change
          - Values change?

## Get area for worst case! 

In [2]:
import xarray as xr

In [6]:
ds=xr.open_mfdataset("/home/yannickh00/ICM/Europe_Task/Results/pvpot_2000.nc", engine="netcdf4", chunks={"valid_time":1e5} )

In [8]:
print(ds)

<xarray.Dataset> Size: 50MB
Dimensions:                        (latitude: 721, longitude: 1440, month: 12)
Coordinates:
    number                         int64 8B ...
  * latitude                       (latitude) float64 6kB 90.0 89.75 ... -90.0
  * longitude                      (longitude) float64 12kB 0.0 0.25 ... 359.8
  * month                          (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    __xarray_dataarray_variable__  (month, latitude, longitude) float32 50MB dask.array<chunksize=(12, 721, 1440), meta=np.ndarray>
